In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from sklearn.model_selection import train_test_split

In [4]:
! pip install keras


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 데이터 읽어오기
train_file = "d:/2nd_project/Data/talk_data/영화주제 대화 말뭉치 라벨링.csv"
data = pd.read_csv(train_file, delimiter=',')
queries = data['text'].tolist()
intents = data['label'].tolist()

from Preprocess2 import Preprocess2
p = Preprocess2(word2index_dic='d:/2nd_project/Data/chatbot_dict_talk.bin', userdic='d:/2nd_project/Data/user_dic.txt')

# 단어 시퀀스 생성
sequences = []

for sentence in queries:
    sentence = str(sentence)
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

# 단어 인덱스 시퀀스 벡터
# 단어 시퀀스 벡터 크기
from GlobalParams import MAX_SEQ_LEN

padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# (105658, 15)
print(padded_seqs.shape)
print(len(intents))  # 105658

# 학습용, 검증용, 테스트용 데이터셋 생성
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# X_train, X_temp, y_train, y_temp = train_test_split(padded_seqs, intents, test_size=0.3, random_state=123)
# X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.7, random_state=123)

# train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(20)
# val_ds = tf.data.Dataset.from_tensor_slices((X_validation, y_validation)).batch(20)
# test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(20)
train_ds = ds.take(len(X_train)).batch(20)
val_ds = ds.skip(len(X_train)).take(len(X_validation)).batch(20)
test_ds = ds.skip(len(X_train) + len(X_validation)).take(len(X_test)).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1  # 전체 단어 개수

# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)
conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(6, name='logits')(dropout_hidden)
predictions = Dense(6, activation=tf.nn.softmax)(logits)

# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
with tf.device('/GPU:0'):
    model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

# 모델 평가(테스트 데이터 셋 이용)
with tf.device('/GPU:0'):
    loss, accuracy = model.evaluate(test_ds, verbose=1)
    print('Accuracy: %f' % (accuracy * 100))
    print('loss: %f' % (loss))

# 모델 저장
model.save('d:/2nd_project/Model/intent_model_0807_h.h5')

print('완료되었습니다.')

(74079, 15)
74079
Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Python38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Python38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Python38\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Python38\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Python38\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "c:\Python38\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Python38\lib\asyncio\base_events.py", line 570, in run_forever
      self._run_once()
    File "c